# 여름데이터 전처리 진행

## 함수형 코드 진행

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
from scipy import stats

# imputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load data
data = pd.read_csv('../data/fall_train.csv')
test = pd.read_csv('../data/fall_test.csv')
test_data = test.copy()

In [3]:
# 변수간의 상관관계가 있는지 확인하고자 하는 함수
def correlation_cal(data, ycol, xcol):
    df = data.copy()
    df = df.dropna(axis = 0)
    select = []
    for names in xcol:
        if df[names].dtypes in ['int64', 'float64']:
            c, p = stats.pearsonr(df[names], df[ycol])
            if p <= 0.05 and abs(c) >= 0.5:
                select.append(names)
    
    return select + [ycol]

In [4]:
# 이상치 탐색하기
def abnormal_data(data, ycol, data_type, Q1, Q2, Q3):
    df = data.copy()
    if data_type == "train":
        Q1 = np.quantile(df[ycol][df[ycol].notnull()], 0.25)
        Q2 = np.quantile(df[ycol][df[ycol].notnull()], 0.5)
        Q3 = np.quantile(df[ycol][df[ycol].notnull()], 0.75)
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df, Q1, Q2, Q3
    
    elif data_type == "test":
        IQR = Q3 - Q1
        df[ycol][(df[ycol] < (Q2 - IQR * 1.5)) | (df[ycol] > (Q2 + IQR * 1.5))] = np.nan
        return df
    
    else:
        print("없는 데이터 타입입니다. 확인하시고 train과 test중 하나를 입력해주세요.")

In [5]:
# imputation 적용하기 : 두 가지 방법을 활용해 적용해두기
# fit한 모델을 그대로 가져와 test셋에도 똑같이 적용하기 위해 return에 포함시킨다.
def imputation_method(method, data, cols, neightbor = 2, weight = "uniform"):
    if method == "mice":
        imputer_mice = IterativeImputer(random_state=42)
        imputer_mice.fit(data[cols])

        # 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
        data[cols] = pd.DataFrame(imputer_mice.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_mice, data
    
    elif method == "knn":
        # KNN을 이용한 자동 대치
        imputer_KNN = KNNImputer(n_neighbors=neightbor, weights=weight)
        imputer_KNN.fit(data[cols])

        # 데이터 변환
        data[cols] = pd.DataFrame(imputer_KNN.transform(data[cols]), columns=data[cols].columns)
        
        # 함수와 데이터 전부 리턴해야 나중에 테스트 셋에서도 사용할 수 있다.
        return imputer_KNN, data
    
    else:
        # 다른 임퓨터는 없어요
        print("그 임퓨터는 지원되지 않습니다. 이름을 확인하거나 소문자로 입력 되었는지 확인해주소.")

In [6]:
# 원하는 변수의 평균값 구해두기
def mean_number(data, needs):
    df = data.copy()
    mean_list = df[needs].mean()
    
    for name in tqdm.tqdm(needs):
        df[name][df[name].isna()] = mean_list[name]
    
    print(df[needs].isna().sum())
    return df, mean_list

In [7]:
# 일사, 일조량 데이터 결측값 대치
def sun_power(selectdata, wantcol, hour, sunrise, sunset, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            ((df[hour] < df[sunrise]) | (df[hour] > df[sunset])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (df[weather].isin(['C'])),
            (df[sunrise] <= df[hour]) & (df[hour] <= df[sunset]) & (~df[weather].isin(['C']))
        ]
        choices = [
            df[name], 
            0, 
            round(df[name][df[weather].isin(['C'])].mean(), 1),
            round(df[name][~df[weather].isin(['C'])].mean(), 1)
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

In [8]:
# 강수량, 강수유무 결측값 대치
def rain_power(selectdata, wantcol, weather):
    df = selectdata.copy()
    for name in tqdm.tqdm(wantcol):
        conditions = [
            df[name].notnull(),
            (df[weather].isin(['R'])),
            (df[weather].isin(['F', 'H'])),
            (~df[weather].isin(['R', 'F', 'H'])),
        ]
        choices = [
            df[name], 
            round(df[name][df[weather].isin(['R'])].mean(), 1),
            round(df[name][df[weather].isin(['F', 'H'])].mean(), 1),
            round(df[name][~df[weather].isin(['R', 'F', 'H'])].mean(), 1),
                  ]
        df[name] = np.select(conditions, choices)
    return df, choices

## 적용 코드

In [9]:
# 기온 이슬점 습도 풍속 대치
data2, mlist = mean_number(data, ['ta', 'td', 'hm', 'ws'])

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 173.66it/s]

ta    0
td    0
hm    0
ws    0
dtype: int64


In [10]:
# 태양과 관련된 변수 대치
data3, param_s = sun_power(data2, ['si', 'ss'], 'hh', 'sunriseh', 'sunseth', 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29.34it/s]


In [11]:
# 비와 관련된 변수 대치
data4, param_r = rain_power(data3, ['rn', 're'], 'ww')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.20it/s]


In [12]:
# 봄이므로 적설량 없다 취급
data4['sn'][data4['sn'].isna()] = 0

In [13]:
data4.columns

Index(['stn', 'year', 'ta', 'td', 'hm', 'ws', 'rn', 're', 'ww', 'ts', 'si',
       'ss', 'sn', 'mm', 'dd', 'hh', 'sunriseh', 'sunrisem', 'sunseth',
       'sunsetm'],
      dtype='object')

In [14]:
# 이상치를 결측치로 전환과 동시에 Q1, Q2, Q3 값 뽑아주기
# test 데이테에서 또한 사용해야함
data5, Q1, Q2, Q3 = abnormal_data(data4, "ts", "train", 0, 0, 0)

In [15]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
data5['ss'] = round(data5['ss'] * 60)

In [16]:
# 필요없는 변수 빼기
data6 = data5[data5.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww', 'Unnamed: 0', 'sn'])]

In [17]:
# 상관계수 절댓값 0.5이상과 p-value가 0.05이하인 변수만 선택
# select_col = correlation_cal(data6, 'ts', list(data6.columns.difference(['ts'])))

In [18]:
# imputation 진행하기
# 연도 제외시키기
tmp = data6.drop(['year'],axis = 1)

# mice, data7 = imputation_method("mice", data6, select_col)
# knn, data8 = imputation_method("knn", data6, select_col)
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(tmp)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
tmp2 = pd.DataFrame(imputer_mice.transform(tmp), columns=tmp.columns)

# 데이터에 mice한 값 집어넣기
data6['ts'] = tmp2['ts']

In [19]:
# 데이터 저장
data6.to_csv('../data/fall_impute.csv', index = False)

===================================================================================================

In [20]:
# 계산식 활용해서 변수 추가
data7 = data6.copy()

In [21]:
# 체감온도(℃)
data7['sensory_temp'] = 13.12 + 0.6215*data7['ta'] - 11.37*(data7['ws']**0.16) + 0.3965*(data7['ws']**0.16)*(data7['ta'])

In [22]:
# 온도에 따른 포화 수증기압(mb)
data7['vapor_pressure'] = 6.11*10**((7.5*data7['ta'])/(237.3 + data7['ta']))

In [23]:
# 현재 대기의 수증기압(mb)
data7['air_pressure'] = data7['hm'] * data7['vapor_pressure'] / 100

In [24]:
# 절대습도(g/m^3)
data7['abs_hm'] = (0.794*data7['air_pressure'])/(1+0.00366*data7['ta'])

In [25]:
# 공기중의 혼합비(mb)
data7['air_mix'] = 0.622 * data7['air_pressure']/(1013.25 - data7['air_pressure'])

In [26]:
# 넷째자리까지 반올림해서 저장
round(data7, 4).to_csv('../data/fall_new_value.csv', index = False)

===================================================================================================

- 여기부터

In [27]:
data['sun'] = 0
data['sun'][data['ww'] == "C"] = 1

In [28]:
data['rain'] = 0
data['rain'][data['ww'] == "R"] = 2
data['rain'][data['ww'].isin(['F', 'H'])] = 1

In [29]:
data['si'][((data['hh'] < data['sunriseh']) | (data['hh'] > data['sunseth'])) & (data['si'].isna())] = 0
data['ss'][((data['hh'] < data['sunriseh']) | (data['hh'] > data['sunseth'])) & (data['ss'].isna())] = 0

In [30]:
# 첫번째 mice
micedata = data[['ta', 'td', 'hm', 'ws', 'sun', 'rain', 'rn', 're', 'si', 'ss', 'mm', 'dd', 'hh']]

In [31]:
# mice
first_imputer_mice = IterativeImputer(random_state=42)
first_imputer_mice.fit(micedata)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
micedata2 = pd.DataFrame(first_imputer_mice.transform(micedata), columns=micedata.columns)

In [32]:
micedata2['ts'] = data['ts']

In [33]:
# mice
imputer_mice = IterativeImputer(random_state=42)
imputer_mice.fit(micedata2)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
micedata3 = pd.DataFrame(imputer_mice.transform(micedata2), columns=micedata2.columns)

In [34]:
micedata3['year'] = data['year']
micedata3['stn'] = data['stn']

In [35]:
# 체감온도(℃)
# micedata3['sensory_temp'] = 13.12 + 0.6215*micedata3['ta'] - 11.37*(micedata3['ws']**0.16) + 0.3965*(micedata3['ws']**0.16)*(micedata3['ta'])

In [36]:
# 온도에 따른 포화 수증기압(mb)
# micedata3['vapor_pressure'] = 6.11*10**((7.5*micedata3['ta'])/(237.3 + micedata3['ta']))

In [37]:
# 현재 대기의 수증기압(mb)
# micedata3['air_pressure'] = micedata3['hm'] * micedata3['vapor_pressure'] / 100

In [38]:
# 절대습도(g/m^3)
# micedata3['abs_hm'] = (0.794*micedata3['air_pressure'])/(1+0.00366*micedata3['ta'])

In [39]:
# 공기중의 혼합비(mb)
# micedata3['air_mix'] = 0.622 * micedata3['air_pressure']/(1013.25 - micedata3['air_pressure'])

In [40]:
micedata3[micedata3.columns.difference(['sun', 'rain'])]

,dd,hh,hm,mm,re,rn,si,ss,stn,ta,td,ts,ws,year
0,1.0,0.0,99.6,8.0,0.0,0.0,0.0,0.0,1,24.2,24.1,24.0,0.8,A
1,1.0,1.0,99.6,8.0,0.0,0.0,0.0,0.0,1,24.0,24.0,24.0,0.6,A
2,1.0,2.0,99.9,8.0,0.0,0.0,0.0,0.0,1,23.8,23.8,23.8,0.7,A
3,1.0,3.0,99.8,8.0,0.0,0.0,0.0,0.0,1,23.7,23.7,23.8,0.7,A
4,1.0,4.0,99.6,8.0,0.0,0.0,0.0,0.0,1,23.8,23.7,23.9,0.6,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110212,31.0,19.0,63.7,10.0,0.0,0.0,0.0,0.0,10,17.9,10.9,15.9,1.4,E
110213,31.0,20.0,67.1,10.0,0.0,0.0,0.0,0.0,10,17.1,11.0,15.0,2.3,E
110214,31.0,21.0,68.9,10.0,0.0,0.0,0.0,0.0,10,16.8,11.0,14.5,2.7,E
110215,31.0,22.0,69.5,10.0,0.0,0.0,0.0,0.0,10,16.0,10.4,14.0,2.4,E


In [41]:
# 넷째자리까지 반올림해서 저장
round(micedata3[micedata3.columns.difference(['sun', 'rain'])], 4).to_csv('../data/fall_mice_twice_train.csv', index = False)

## Test data

In [42]:
# 값 대치
for name in tqdm.tqdm(['ta', 'td', 'hm', 'ws']):
        test_data[name][test_data[name].isna()] = mlist[name]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 227.53it/s]


In [43]:
# 태양관련 변수 대입
for name in tqdm.tqdm(['si', 'ss']):
    conditions = [
        test_data[name].notnull(),
        ((test_data['hh'] < test_data['sunriseh']) | (test_data['hh'] > test_data['sunseth'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (test_data['ww'].isin(['C'])),
        (test_data['sunriseh'] <= test_data['hh']) & (test_data['hh'] <= test_data['sunseth']) & (~test_data['ww'].isin(['C']))
    ]
    param_s[0] = test_data[name]
    test_data[name] = np.select(conditions, param_s)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 137.00it/s]


In [44]:
# 비 관련 변수 대입
for name in tqdm.tqdm(['rn', 're']):
    conditions = [
        test_data[name].notnull(),
        (test_data['ww'].isin(['R'])),
        (test_data['ww'].isin(['F', 'H'])),
        (~test_data['ww'].isin(['R', 'F', 'H'])),
    ]
    param_r[0] = test_data[name]
    test_data[name] = np.select(conditions, param_r)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 219.56it/s]


In [45]:
# 적설량 0으로 통일
# 여름이므로 적설량 없다 취급
test_data['sn'][test_data['sn'].isna()] = 0

In [46]:
# 시간단위를 분 단위로 바꿔주기(이 코드는 한번만 돌려야지 잘 적용된다.)
test_data['ss'] = round(test_data['ss'] * 60)

In [47]:
# 필요없는 변수 빼기
test_data2 = test_data[test_data.columns.difference(['sunriseh', 'sunrisem', 'sunseth', 'sunsetm', 'ww'])]

In [48]:
# 데이터 저장
test_data2.to_csv('../data/fall_test_complete.csv', index = False)

===================================================================================================

In [49]:
# test
test_data3 = test_data2.copy()

In [50]:
# 체감온도(℃)
test_data3['sensory_temp'] = 13.12 + 0.6215*test_data3['ta'] - 11.37*(test_data3['ws']**0.16) + 0.3965*(test_data3['ws']**0.16)*(test_data3['ta'])

In [51]:
# 온도에 따른 포화 수증기압(mb)
test_data3['vapor_pressure'] = 6.11*10**((7.5*test_data3['ta'])/(237.3 + test_data3['ta']))

In [52]:
# 현재 대기의 수증기압(mb)
test_data3['air_pressure'] = test_data3['hm'] * test_data3['vapor_pressure'] / 100

In [53]:
# 절대습도(g/m^3)
test_data3['abs_hm'] = (0.794*test_data3['air_pressure'])/(1+0.00366*test_data3['ta'])

In [54]:
# 공기중의 혼합비(mb)
test_data3['air_mix'] = 0.622 * test_data3['air_pressure']/(1013.25 - test_data3['air_pressure'])

In [55]:
# 넷째자리까지 반올림해서 저장
round(test_data3, 4).to_csv('../data/fall_new_test_value.csv', index = False)

===================================================================================================

In [56]:
test['sun'] = 0
test['sun'][test['ww'] == "C"] = 1

In [57]:
test['rain'] = 0
test['rain'][test['ww'] == "R"] = 2
test['rain'][test['ww'].isin(['F', 'H'])] = 1

In [58]:
# 첫번째 mice
testmicedata = test[['ta', 'td', 'hm', 'ws', 'sun', 'rain', 'rn', 're', 'si', 'ss', 'mm', 'dd', 'hh']]

In [59]:
testmicedata2 = pd.DataFrame(first_imputer_mice.transform(testmicedata), columns=testmicedata.columns)

In [60]:
testmicedata2['year'] = test['year']
testmicedata2['stn'] = test['stn']

In [61]:
testmicedata3 = testmicedata2[testmicedata2.columns.difference(['sun', 'rain'])]

In [62]:
# 체감온도(℃)
# testmicedata2['sensory_temp'] = 13.12 + 0.6215*testmicedata2['ta'] - 11.37*(testmicedata2['ws']**0.16) + 0.3965*(testmicedata2['ws']**0.16)*(testmicedata2['ta'])

In [63]:
# 온도에 따른 포화 수증기압(mb)
# testmicedata2['vapor_pressure'] = 6.11*10**((7.5*testmicedata2['ta'])/(237.3 + testmicedata2['ta']))

In [64]:
# 현재 대기의 수증기압(mb)
# testmicedata2['air_pressure'] = testmicedata2['hm'] * testmicedata2['vapor_pressure'] / 100

In [65]:
# 절대습도(g/m^3)
# testmicedata2['abs_hm'] = (0.794*testmicedata2['air_pressure'])/(1+0.00366*testmicedata2['ta'])

In [66]:
# 공기중의 혼합비(mb)
# testmicedata2['air_mix'] = 0.622 * testmicedata2['air_pressure']/(1013.25 - testmicedata2['air_pressure'])

In [67]:
# 넷째자리까지 반올림해서 저장
round(testmicedata2, 4).to_csv('../data/fall_mice_twice_test.csv', index = False)